# Raw RAG 05: Pydantic is All You Need (Really!)

## Introduction

In the world of Retrieval-Augmented Generation (RAG) systems, structured data plays a crucial role in ensuring accuracy and facilitating seamless integration with other components. While there are numerous libraries and frameworks available for handling JSON responses from Language Models (LLMs), this notebook focuses on a lightweight and flexible approach using only the Pydantic and json libraries.
Why Pydantic?

* **Type Safety**: Pydantic provides runtime type checking and data validation.
* **Simplicity**: It offers a straightforward way to define data models using Python type annotations.
* **Flexibility**: Easily extensible to accommodate complex data structures.
* **Performance**: Pydantic is optimized for speed, making it suitable for high-performance applications.

## Objectives
In this notebook, we will:

* Introduce basic Pydantic models for structuring LLM responses
* Demonstrate how to parse JSON responses into Pydantic models
* Explore advanced Pydantic features for handling complex data structures
* Discuss best practices for integrating Pydantic into your RAG pipeline

By the end of this notebook, you'll have a solid understanding of how to use Pydantic for JSON parsing in your RAG system. This knowledge will empower you to adapt and extend these concepts to fit your specific needs, ensuring robust and type-safe data handling throughout your project.
Let's dive in and explore the power of Pydantic in RAG systems!

Reference:
- [Pydantic](https://pydantic-docs.helpmanual.io/)
- [Video: Pydantic is All You Need](https://www.youtube.com/watch?v=yj-wSRJwrrc)

In [1]:
# Install needed packages

%pip install openai pydantic python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load the environment variables from the .env file

from dotenv import load_dotenv
import os

dotenv_path = ".env"
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
from openai import OpenAI
from openai.types.chat.completion_create_params import ResponseFormat

client = OpenAI()

## Option A: Direct JSON Response Request

Leveraging the built-in JSON support of the ChatGPT API, we can streamline our process by directly requesting responses in JSON format. This approach eliminates the need for additional parsing steps, potentially reducing processing time and complexity.

Key points:

* Include the phrase "json_response_format" in your query
* Set `response_format={"type": 'json_object'}` in the API call
* Ensures a structured, easily parseable response

By utilizing this method, we can efficiently obtain structured data from the LLM, setting the stage for seamless integration with our RAG system.

In [4]:
# Note: you must include the word "json_response_format" in query and response_format={"type": 'json_object'} in function to get the response in json format

full_query = f"""You are a customer relations manager at a hotel. A guest named John Smith is staying in room 101 and has requested extra towels. Write a message to the housekeeping staff instructing them to fulfill the request.

json_response_format: room_number, guest_name, request

"""

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You help user with their request.",
        },
        {"role": "user", "content": full_query},
    ],
    response_format={"type": 'json_object'},
    model="gpt-4-turbo",
    temperature=0,
)

resp = response.choices[0].message.content

print(resp)

{
  "room_number": "101",
  "guest_name": "John Smith",
  "request": "extra towels"
}


# Option B: ChatGPT API Function Call for JSON Responses

While Option A offers simplicity, it may lack robustness when dealing with inconsistent field types. The ChatGPT API's function call feature provides a more structured and reliable alternative for obtaining JSON responses.

Key advantages:

* Enhanced control over response structure
* Improved consistency in field types
* Reduced error potential in complex scenarios

By leveraging function calls, we can define precise schemas for our desired outputs, ensuring that the LLM adheres to our specified format. This method is particularly valuable when working with intricate data structures or when maintaining strict type consistency is crucial for downstream processing in your RAG pipeline.

In [5]:
# Define the function tools and output schema

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_guest_request",
            "description": "Get the guest request",
            "parameters": {
                "type": "object",
                "properties": {
                    "room_number": {
                        "type": "string",
                        "description": "guest room number",
                    },
                    "guest_name": {
                        "type": "string",
                        "description": "guest name",
                    },
                    "request": {
                        "type": "string",
                        "description": "guest request",
                    },
                },
                "required": ["room_number", "request"],
            },
        }
    },
]

In [6]:
# Call the function

new_query = f"""You are a customer relations manager at a hotel. A guest named John Smith is staying in room 101 and has requested extra towels. Write a message to the housekeeping staff instructing them to fulfill the request.

"""

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You help user with their request.",
        },
        {"role": "user", "content": new_query},
    ],
    model="gpt-4-turbo",
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "get_guest_request"}}
)

function_call_resp = response.choices[0].message

print(function_call_resp)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_QltSm9v9215FvsZXtaueBM9q', function=Function(arguments='{"room_number":"101","guest_name":"John Smith","request":"extra towels"}', name='get_guest_request'), type='function')])


In [7]:
# Let's build a function to extract and parse the arguments from the response

import json

def extract_and_parse_arguments(message) -> dict:
    if message.tool_calls and len(message.tool_calls) > 0:
        first_tool_call = message.tool_calls[0]
        arguments_str = first_tool_call.function.arguments
        return json.loads(arguments_str)
    else:
        return {}

In [8]:
parsed_args = extract_and_parse_arguments(function_call_resp)

print(parsed_args)

{'room_number': '101', 'guest_name': 'John Smith', 'request': 'extra towels'}


## Option C: Pydantic for JSON Schema Generation

While manually crafting JSON schemas is feasible, it can become unwieldy and error-prone, especially for complex structures. Pydantic offers a more elegant and maintainable solution for schema creation.

Key benefits:

* Improved readability and maintainability
* Automatic type validation and error checking
* Seamless conversion to JSON schema format

By leveraging Pydantic's intuitive class-based model definitions, we can create clear, structured schemas that are easy to understand and modify. This approach not only reduces the likelihood of errors but also enhances code quality and developer productivity in your RAG system.

In [9]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, Dict, List, Any

# Create a Pydantic model to represent the guest request
class GuestRequest(BaseModel):
    model_config = ConfigDict(
        title="extract user request detail",
        description="Based on the user request, return the guest request details.",
    )
    room_number: str = Field(..., description="guest room number")
    guest_name: Optional[str] = Field(None, description="guest name")
    request: str = Field(..., description="guest request")

In [10]:
from utils import MessageParser, SchemaConverter

message_parser = MessageParser()
schema_converter = SchemaConverter()

pydantic_tool = schema_converter.pydantic_to_function_schema(GuestRequest)
print(json.dumps(pydantic_tool, indent=2))

{
  "type": "function",
  "function": {
    "name": "extract_user_request_detail",
    "description": "",
    "parameters": {
      "type": "object",
      "properties": {
        "room_number": {
          "type": "string",
          "description": "guest room number"
        },
        "guest_name": {
          "type": "string",
          "description": "guest name"
        },
        "request": {
          "type": "string",
          "description": "guest request"
        }
      },
      "required": [
        "room_number",
        "request"
      ]
    }
  }
}


In [11]:
new_query = "You are a customer relations manager at a hotel. A guest named John Smith is staying in room 101 and has requested extra towels. Write a message to the housekeeping staff instructing them to fulfill the request."

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You help user with their request.",
        },
        {"role": "user", "content": new_query},
    ],
    model="gpt-4-turbo",
    tools=[pydantic_tool],
    tool_choice={
        "type": "function",
        "function": {"name": "extract_user_request_detail"},
    },
)

pydantic_resp = response.choices[0].message

print(pydantic_resp)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AmSkDHwAPh37iLHVOmUga0kA', function=Function(arguments='{"room_number":"101","guest_name":"John Smith","request":"extra towels"}', name='extract_user_request_detail'), type='function')])


In [12]:
parsed_args = message_parser.extract_and_parse_arguments(pydantic_resp)

print(parsed_args)

{'room_number': '101', 'guest_name': 'John Smith', 'request': 'extra towels'}


## Conclusion: Unlocking Precision and Potential in RAG Systems

Throughout this notebook, we've explored various methods to structure and control LLM outputs using JSON parsing techniques. By leveraging direct JSON responses, API function calls, and Pydantic models, we've demonstrated how to obtain precisely formatted data from language models.
Key takeaways:

* Structured outputs enhance accuracy and integration in RAG systems
* JSON parsing techniques provide fine-grained control over LLM responses
* Pydantic offers a powerful, type-safe approach to defining and validating data structures

The ability to shape LLM outputs into well-defined objects opens up new possibilities:

* Seamless integration with databases and APIs
* Enhanced data validation and error handling
* Simplified post-processing and analysis workflows

By mastering these techniques, you're not just parsing JSON – you're unlocking the full potential of your RAG system. With structured, reliable data at your fingertips, you can confidently build more sophisticated applications, integrate advanced tools, and push the boundaries of what's possible with AI-augmented systems.

Remember, the key to innovation lies in control and precision. As you continue to develop your RAG pipeline, consider how these parsing techniques can be leveraged to create more powerful, flexible, and reliable LLM-driven solutions.